In [45]:
import selenium
from bs4 import BeautifulSoup
import time
import pandas as pd


# Importing selenium webdriver
from selenium import webdriver

# Importing Exceptions
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException, ElementNotVisibleException

# importing requests
import requests

#importing regex
import re

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [46]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

In [47]:
# Getting the webpage of mentioned url
url = "https://flight.easemytrip.com"
driver.get(url)
time.sleep(3)

In [48]:
# searching flights
search_btn = driver.find_element_by_xpath('//input[@value="Search"]')
search_btn.click()
time.sleep(1)

In [49]:
# creating a list of cities to see flight tickets
cities=['Delhi','Bangalore','Mumbai','Kolkata','Goa', 'Hyderabad', 'Chennai']

In [50]:
# Creating empty lists to store the values
source_city = []
destination_city = []
airline_name=[]
departure_time = []
arrival_time = []
duration=[]
stops = []
source = []
destination = []
loc=[]
price=[]

In [51]:
for a in cities:
    for b in cities:
        if a!=b:            
            source_city = driver.find_element_by_xpath('//input[@id="FromSector_show"]')
            time.sleep(2)
            source_city.clear() 
            source_city.send_keys(a)
            time.sleep(2)
            
            destination_city = driver.find_element_by_xpath('//input[@id="Editbox13_show"]')
            time.sleep(2)
            destination_city.click()
            destination_city.clear()
            time.sleep(2)
            destination_city.send_keys(b)
            time.sleep(2)
            
            try:
                search_button = driver.find_element_by_xpath("//button[@id='btnSrch']")
                wait = WebDriverWait(driver, 10)
                #wait.until(EC.visibility_of(search_button))
                search_button.click()
            except NoSuchElementException:
                    pass
            time.sleep(1)
            
            #scrolling the page
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
                        
           #fetching name of airline
            try:
                for i in driver.find_elements_by_xpath("//span[@class='txt-r4 ng-binding']"):
                    airline_name.append(i.text)
            except NoSuchElementException:
                    pass
            
            #time of depart
            try:
                for i in driver.find_elements_by_xpath("//div[@class='col-md-2 col-sm-2 col-xs-4 top5 ']/span[1]"):
                    departure_time.append(i.text)
            except NoSuchElementException:
                    pass
            
            #time of arrival
            try:
                for i in driver.find_elements_by_xpath("//div[@class='col-md-2 col-sm-2 col-xs-3 top5']//span[1]"):
                    arrival_time.append(i.text)
                    #arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                    pass
            
            #duration
            try:
                for i in driver.find_elements_by_xpath("//div[@class='col-md-2 col-sm-2 col-xs-5 non-st']/span[1]"):
                    duration.append(i.text)
            except NoSuchElementException:
                    pass
            
            #number of stops
            try:
                for i in driver.find_elements_by_xpath("//div[@class='col-md-2 col-sm-2 col-xs-5 non-st']/span[2]"):
                    stops.append(i.text)
            except NoSuchElementException:
                    pass
            
            #locations
            try:
                for i in driver.find_elements_by_xpath("//span[@class='txt-r3-n ng-binding']"):
                    loc.append(i.text)
            except NoSuchElementException:
                    pass
                
                        
            #fetching prices
            try:
                for i in driver.find_elements_by_xpath("//div[@class='col-md-8 col-sm-8 col-xs-9 txt-r6-n exPrc ng-binding']"):
                    price.append(i.text)
            except NoSuchElementException:
                    pass

In [52]:
# Checking lengths of all features
print(len(airline_name),len(departure_time),len(arrival_time),len(duration),len(stops),len(price))

3673 3673 3673 3673 3673 3673


In [53]:
source = loc[0:len(loc):2]
destination = loc[1:len(loc):2]

In [54]:
df = pd.DataFrame({"Airline Name":airline_name, "Source":source, "Destination":destination, "Departure Time":departure_time, "Arrival Time":arrival_time[:4500], "Duration":duration, "Total Stops":stops, "Price":price})
df

,Airline Name,Source,Destination,Departure Time,Arrival Time,Duration,Total Stops,Price
0,Indigo,Delhi,Bangalore,03:50,06:40,02h 50m,non-stop,"9,419"
1,Indigo,Delhi,Bangalore,22:55,01:45,02h 50m,non-stop,"9,419"
2,GO FIRST,Delhi,Bangalore,05:50,08:30,02h 40m,non-stop,"9,261"
3,AirAsia,Delhi,Bangalore,21:40,00:30,02h 50m,non-stop,"9,419"
4,GO FIRST,Delhi,Bangalore,20:30,23:20,02h 50m,non-stop,"9,261"
...,...,...,...,...,...,...,...,...
3668,Vistara,Chennai,Hyderabad,10:35,09:25,22h 50m,1-stop,"22,528"
3669,Vistara,Chennai,Hyderabad,17:15,09:25,16h 10m,1-stop,"23,263"
3670,Air India,Chennai,Hyderabad,16:55,23:45,06h 50m,1-stop,
3671,Air India,Chennai,Hyderabad,20:15,11:50,15h 35m,1-stop,


In [58]:
df.tail()

,Airline Name,Source,Destination,Departure Time,Arrival Time,Duration,Total Stops,Price
3668,Vistara,Chennai,Hyderabad,10:35,09:25,22h 50m,1-stop,"22,528"
3669,Vistara,Chennai,Hyderabad,17:15,09:25,16h 10m,1-stop,"23,263"
3670,Air India,Chennai,Hyderabad,16:55,23:45,06h 50m,1-stop,
3671,Air India,Chennai,Hyderabad,20:15,11:50,15h 35m,1-stop,
3672,Air India,Chennai,Hyderabad,20:15,19:30,23h 15m,1-stop,


In [55]:
df.duplicated().sum()

58

In [56]:
# Saving the data into excel file
df.to_excel("Flight_Dataset.xlsx")

In [57]:
df.to_csv('Flight_price_data', index=False)